<a href="https://colab.research.google.com/github/chosunghyun18/pythonproj_for_HML_and_financial/blob/main/Finacial%20engineering%20with%20python/S%26P500_croling_jsontype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title import library
import pandas as pd
from urllib.request import urlopen
import json
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import bs4

In [ ]:

def historical_index_naver(index_cd, start_date='', end_date='', page_n=1, last_page=0):
    
    if start_date:   # start_date가 있으면
        start_date = date_format(start_date)   # date 포맷으로 변환
    else:    # 없으면
        start_date = dt.date.today()   # 오늘 날짜를 지정
    if end_date:   
        end_date = date_format(end_date)   
    else:   
        end_date = dt.date.today()  
        
        
    naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)
    
    source = urlopen(naver_index).read()   # 지정한 페이지에서 코드 읽기
    source = bs4.BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
    
    dates = source.find_all('td', class_='date')   # <td class="date">태그에서 날짜 수집   
    prices = source.find_all('td', class_='number_1')   # <td class="number_1">태그에서 지수 수집
    
    for n in range(len(dates)):
    
        if dates[n].text.split('.')[0].isdigit():
            
            # 날짜 처리
            this_date = dates[n].text
            this_date= date_format(this_date)
            
            if this_date <= end_date and this_date >= start_date:   
            # start_date와 end_date 사이에서 데이터 저장
                # 종가 처리
                this_close = prices[n*4].text   # prices 중 종가지수인 0,4,8,...번째 데이터 추출
                this_close = this_close.replace(',', '')
                this_close = float(this_close)

                # 딕셔너리에 저장
                historical_prices[this_date] = this_close
                
            elif this_date < start_date:   
            # start_date 이전이면 함수 종료
                return historical_prices              
            
    # 페이지 네비게이션
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        # 마지막페이지 주소 추출
        last_page = last_page.split('&')[1]   # & 뒤의 page=506 부분 추출
        last_page = last_page.split('=')[1]   # = 뒤의 페이지번호만 추출
        last_page = int(last_page)   # 숫자형 변수로 변환
        
    # 다음 페이지 호출
    if page_n < last_page:   
        page_n = page_n + 1   
        historical_index_naver(index_cd, start_date, end_date, page_n, last_page)   
        
    return historical_prices

In [ ]:
import json
symbol ='NII@NI225'
page = 1 
def read_json(d, symbol, page=1):
    url = 'https://finance.naver.com/world/worldDayListJson.nhn?symbol='+symbol+'&fdtc=0&page='+str(page)
    raw = urlopen(url)
    data = json.load(raw)
    
    for n in range(len(data)):
        date = pd.to_datetime(data[n]['xymd']).date()
        price = float(data[n]['clos'])
        d[date] = price
        
    if len(data) == 10 and page<3:
        page += 1
        read_json(d, symbol, page)
        
    return (d)

In [ ]:

def date_format(d=''):
    if d != '':
        this_date = pd.to_datetime(d).date()
    else:
        this_date = pd.Timestamp.today().date()   # 오늘 날짜를 지정
    return (this_date)

In [ ]:
index_cd = 'KPI200'
historical_prices = dict()
historical_index_naver(index_cd, '2020-4-1', '2020-4-4')
historical_prices

In [ ]:
historical_index = pd.Series()
historical_index = read_json(historical_index, symbol, page)
historical_index.head(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


2020-07-31    21710.00
2020-07-30    22339.23
2020-07-29    22397.11
dtype: float64

In [ ]:
indices = {
    'SPI@SPX' : 'S&P 500', 
    'NAS@NDX' : 'Nasdaq 100', 
    'NII@NI225' : 'Nikkei 225'
}

In [ ]:
historical_indices = dict()
for key, value in indices.items(): 
    print (key, value)
    s = dict()
    s = read_json(s, key, 1)
    historical_indices[value] = s    
prices_df = pd.DataFrame(historical_indices)

SPI@SPX S&P 500
NAS@NDX Nasdaq 100
NII@NI225 Nikkei 225


In [ ]:
prices_df.tail(3)

,S&P 500,Nasdaq 100,Nikkei 225
2020-06-19,3097.74,10008.64,22478.79
2020-07-03,NaN,NaN,22306.48
2020-06-18,NaN,NaN,22355.46


In [ ]:
def index_global(d,symbol,start_date='',end_date='',page=1):
    end_date = date_format(end_date)
    if start_date == '':
        start_date = end_date - pd.DateOffset(months =1)
    start_date = date_format(start_date)

    url = 'https://finance.naver.com/world/worldDayListJson.nhn?symbol=' + symbol +'&fdtc=0&page='+str(page)

    raw = urlopen(url)
    data = json.load(raw)

    if len(data) > 0 :
     
        for n in range(len(data)):
            date = pd.to_datetime(data(n)['xymd']).date()

            if date <= end_date and date >= start_date:
  
              price = float(data[n]['clos'])
          
              d[date] = price

            elif date <start_date:


              return (d)

        if len(data) == 10:
          page += 1
          index_global(d,symbol,start_date,end_date,page)

    return (d)





In [ ]:
historical_index = dict()
start_date = '2020-06-01'
end_date = '2020-06-03'
for key, value in indices.items():
    s = dict()
    s = index_global(s,key,start_date)
    historical_indices[value] = s
prices_df = pd.DataFrame(historical_indes)
